# EXAMPLE - 5

**Tasks :- NER tagging, POS tagging**

**Tasks Description**

``NER`` :-This is a Named Entity Recognition task where individual words of the sentence are tagged with an entity label it belongs to. The words which don't belong to any entity label are simply labeled as "O".

``POS`` :- This is a Part of Speech tagging task. A part of speech is a category of words that have similar grammatical properties. Each word of the sentence is tagged with the part of speech label it belongs to. The words which don't belong to any part of speech label are simply labeled as "O".

**Conversational Utility** :-  In conversational AI context, determining the syntactic parts of the sentence can help in extracting noun-phrases or important keyphrases from the sentence.

**Data** :- In this example, we are using the <a href="https://www.clips.uantwerpen.be/conll2003/ner/">coNLL 2003</a> data which is BIO tagged format with the POS and NER tags separated by space.

The data is already present in ``coNLL_data`` directory.

# Step - 1: Transforming data

Raw data is in BIO tagged format with the POS and NER tags separated by space.

We already provide a sample transformation function ``coNLL_ner_pos_to_tsv`` to convert this data to required tsv format. 

Running data transformations will save the required train, dev and test tsv data files under ``data`` directory in root of library. For more details on the data transformation process, refer to <a href="https://multi-task-nlp.readthedocs.io/en/latest/data_transformations.html">data transformations</a> in documentation.

The transformation file should have the following details which is already created ``transform_file_conll.yml``.

```
transform1:
  transform_func: coNLL_ner_pos_to_tsv
  read_file_names:
    - coNLL_train.txt
    - coNLL_testa.txt
    - coNLL_testb.txt
  read_dir: coNLL_data
  save_dir: ../../data
 ```
 Following command can be used to run the data transformation for the tasks.

In [1]:
!python ./../data_transformations.py \
    --transform_file 'transform_file_conll.yml'

/home/tiendat/anaconda3/envs/thesis/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
loading file vocab.txt from cache at /home/tiendat/.cache/huggingface/hub/models--dmis-lab--biobert-base-cased-v1.2/snapshots/67c9c25b46986521ca33df05d8540da1210b3256/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /home/tiendat/.cache/huggingface/hub/models--dmis-lab--biobert-base-cased-v1.2/snapshots/67c9c25b46986521ca33df05d8

# Step -2 Data Preparation

For more details on the data preparation process, refer to <a href="https://multi-task-nlp.readthedocs.io/en/latest/training.html#running-data-preparation">data preparation</a> in documentation.

Defining tasks file for training single model for entailment task. The file is already created at ``tasks_file_conll.yml``
```
conllner:
  model_type: BERT
  config_name: bert-base-uncased
  dropout_prob: 0.2
  label_map_or_file: ../../data/ner_coNLL_train_label_map.joblib
  metrics:
  - seqeval_f1_score
  - seqeval_precision
  - seqeval_recall
  loss_type: NERLoss
  task_type: NER
  file_names:
  - ner_coNLL_train.tsv
  - ner_coNLL_testa.tsv
  - ner_coNLL_testb.tsv

conllpos:
    model_type: BERT
    config_name: bert-base-uncased
    dropout_prob: 0.2
    label_map_or_file: ../../data/pos_coNLL_train_label_map.joblib
    metrics:
    - seqeval_f1_score
    - seqeval_precision
    - seqeval_recall
    loss_type: NERLoss
    task_type: NER
    file_names:
    - pos_coNLL_train.tsv
    - pos_coNLL_testa.tsv
    - pos_coNLL_testb.tsv
```

In [13]:
!python ../data_preparation.py \
    --task_file './tasks_file_SRL_DEPENDENCY_GRAMMAR.yml' \
    --data_dir 'content/data/' \
    --max_seq_len 50
    
# !python ./../data_preparation.py \
#     --task_file 'tasks_file_SRL_DEPENDENCY_GRAMMAR.yml' \
#     --data_dir 'content/data' \
#     --max_seq_len 50

/home/tiendat/anaconda3/envs/thesis/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
loading file vocab.txt from cache at /home/tiendat/.cache/huggingface/hub/models--dmis-lab--biobert-base-cased-v1.2/snapshots/67c9c25b46986521ca33df05d8540da1210b3256/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /home/tiendat/.cache/huggingface/hub/models--dmis-lab--biobert-base-cased-v1.2/snapshots/67c9c25b46986521ca33df05d8

# Step -3 Running Training

In [ ]:
!python ../train.py \
    --data_dir 'content/data/bert-base-uncased_prepared_data' \
    --task_file './tasks_file_SRL_DEPENDENCY_GRAMMAR.yml' \
    --out_dir 'conll_ner_pos_bert_base' \
    --epochs 10 \
    --train_batch_size 32 \
    --eval_batch_size 32 \
    --grad_accumulation_steps 1 \
    --log_per_updates 50 \
    --max_seq_len 50 \
    --eval_while_train \
    --test_while_train \
    --silent

# Step - 4 Infering

You can import and use the ``inferPipeline`` to get predictions for the required tasks.
The trained model and maximum sequence length to be used needs to be specified.

For knowing more details about infering, refer to <a href="https://multi-task-nlp.readthedocs.io/en/latest/infering.html">infer pipeline</a> in documentation.

In [ ]:
import sys
sys.path.insert(1, '../../')
from infer_pipeline import inferPipeline